<br><br><br><br><br>

# Awkward datasets

<br><br><br><br><br>

<br><br><br><br><br>

It's not uncommon for data to be non-rectangular. Jagged ("ragged") arrays, cross-references, trees, and graphs are frequently encountered, but difficult to cast as Numpy arrays or Pandas DataFrames.

<br>

**Let's start with NASA's exoplanet database:** each star can have an arbitrary number of planets (jagged array).

<br><br><br><br><br>

In [1]:
import pandas

# NASA provides this dataset as a CSV file, which suggests a rectangular table: one row per planet.
exoplanets = pandas.read_csv("data/nasa-exoplanets.csv")
exoplanets

,loc_rowid,pl_hostname,pl_letter,pl_name,pl_discmethod,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,...,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_disc,st_glon,st_glat,st_radv,st_radverr1,st_radverr2,st_radvlim
0,1,11 Com,b,11 Com b,Radial Velocity,326.030000,0.320000,-0.320000,0.0,1.290000,...,NaN,NaN,NaN,2007,264.140892,78.280522,43.37,0.21,-0.21,0.0
1,2,11 UMi,b,11 UMi b,Radial Velocity,516.219970,3.200000,-3.200000,0.0,1.530000,...,NaN,NaN,0.0,2009,108.718982,41.044516,-17.52,0.15,-0.15,0.0
2,3,14 And,b,14 And b,Radial Velocity,185.840000,0.230000,-0.230000,0.0,0.830000,...,NaN,NaN,NaN,2008,106.411558,-21.050610,-59.73,0.31,-0.31,0.0
3,4,14 Her,b,14 Her b,Radial Velocity,1773.400020,2.500000,-2.500000,0.0,2.930000,...,NaN,NaN,0.0,2002,69.170376,46.944999,NaN,NaN,NaN,NaN
4,5,16 Cyg B,b,16 Cyg B b,Radial Velocity,798.500000,1.000000,-1.000000,0.0,1.660000,...,NaN,NaN,0.0,1996,83.336460,13.204352,-28.10,0.10,-0.10,0.0
5,6,18 Del,b,18 Del b,Radial Velocity,993.300000,3.200000,-3.200000,0.0,2.600000,...,NaN,NaN,NaN,2008,58.691434,-21.974824,4.35,0.15,-0.15,0.0
6,7,1RXS J160929.1-210524,b,1RXS J160929.1-210524 b,Imaging,NaN,NaN,NaN,NaN,330.000000,...,100.0,-100.0,0.0,2008,352.890560,22.030427,NaN,NaN,NaN,NaN
7,8,24 Boo,b,24 Boo b,Radial Velocity,30.350600,0.007800,-0.007700,0.0,0.190000,...,NaN,NaN,0.0,2018,90.162223,60.643666,-7.77,0.15,-0.15,0.0
8,9,24 Sex,b,24 Sex b,Radial Velocity,452.800000,2.100000,-4.500000,0.0,1.333000,...,NaN,NaN,NaN,2010,245.085309,44.715734,7.29,0.18,-0.18,0.0
9,10,24 Sex,c,24 Sex c,Radial Velocity,883.000000,32.400000,-13.800000,0.0,2.080000,...,NaN,NaN,NaN,2010,245.085309,44.715734,7.29,0.18,-0.18,0.0


In [2]:
# Quite a few planets in this table have the same star ("host") name.

numplanets = exoplanets.groupby("pl_hostname").size()
numplanets[numplanets > 1]

pl_hostname
24 Sex                 2
47 UMa                 3
55 Cnc                 5
61 Vir                 3
BD+20 2457             2
BD-06 1339             2
BD-08 2823             2
CoRoT-20               2
CoRoT-24               2
CoRoT-7                2
EPIC 220674823         2
GJ 1132                2
GJ 1148                2
GJ 163                 3
GJ 273                 2
GJ 3138                3
GJ 317                 2
GJ 3293                4
GJ 3323                2
GJ 3998                2
GJ 581                 3
GJ 667 C               5
GJ 676 A               4
GJ 832                 2
GJ 876                 4
GJ 9827                3
HAT-P-11               2
HAT-P-13               2
HAT-P-17               2
HAT-P-44               2
                      ..
Kepler-94              2
Kepler-953             2
Kepler-967             2
Kepler-968             2
Kepler-969             2
Kepler-97              2
Kepler-990             2
LHS 1140               2
LkCa 15      

In [3]:
# Use Pandas's MultiIndex to represent a sparse, 2D index (stars × planets without missing values).

exoplanets.index = pandas.MultiIndex.from_arrays([exoplanets["pl_hostname"], exoplanets["pl_letter"]])
exoplanets.index.names = ["star", "planet"]
exoplanets

,,loc_rowid,pl_hostname,pl_letter,pl_name,pl_discmethod,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,...,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_disc,st_glon,st_glat,st_radv,st_radverr1,st_radverr2,st_radvlim
star,planet,,,,,,,,,,,,,,,,,,,,,
11 Com,b,1,11 Com,b,11 Com b,Radial Velocity,326.030000,0.320000,-0.320000,0.0,1.290000,...,NaN,NaN,NaN,2007,264.140892,78.280522,43.37,0.21,-0.21,0.0
11 UMi,b,2,11 UMi,b,11 UMi b,Radial Velocity,516.219970,3.200000,-3.200000,0.0,1.530000,...,NaN,NaN,0.0,2009,108.718982,41.044516,-17.52,0.15,-0.15,0.0
14 And,b,3,14 And,b,14 And b,Radial Velocity,185.840000,0.230000,-0.230000,0.0,0.830000,...,NaN,NaN,NaN,2008,106.411558,-21.050610,-59.73,0.31,-0.31,0.0
14 Her,b,4,14 Her,b,14 Her b,Radial Velocity,1773.400020,2.500000,-2.500000,0.0,2.930000,...,NaN,NaN,0.0,2002,69.170376,46.944999,NaN,NaN,NaN,NaN
16 Cyg B,b,5,16 Cyg B,b,16 Cyg B b,Radial Velocity,798.500000,1.000000,-1.000000,0.0,1.660000,...,NaN,NaN,0.0,1996,83.336460,13.204352,-28.10,0.10,-0.10,0.0
18 Del,b,6,18 Del,b,18 Del b,Radial Velocity,993.300000,3.200000,-3.200000,0.0,2.600000,...,NaN,NaN,NaN,2008,58.691434,-21.974824,4.35,0.15,-0.15,0.0
1RXS J160929.1-210524,b,7,1RXS J160929.1-210524,b,1RXS J160929.1-210524 b,Imaging,NaN,NaN,NaN,NaN,330.000000,...,100.0,-100.0,0.0,2008,352.890560,22.030427,NaN,NaN,NaN,NaN
24 Boo,b,8,24 Boo,b,24 Boo b,Radial Velocity,30.350600,0.007800,-0.007700,0.0,0.190000,...,NaN,NaN,0.0,2018,90.162223,60.643666,-7.77,0.15,-0.15,0.0
24 Sex,b,9,24 Sex,b,24 Sex b,Radial Velocity,452.800000,2.100000,-4.500000,0.0,1.333000,...,NaN,NaN,NaN,2010,245.085309,44.715734,7.29,0.18,-0.18,0.0


In [4]:
# Simplify the table to show 5 star attributes and 5 planet attributes. Star attributes are repeated.

df = exoplanets[["ra", "dec", "st_dist", "st_mass", "st_rad", "pl_orbsmax", "pl_orbeccen", "pl_orbper", "pl_bmassj", "pl_radj"]]
df.columns = pandas.MultiIndex.from_arrays([["star"] * 5 + ["planet"] * 5,
    ["right asc. (deg)", "declination (deg)", "distance (pc)", "mass (solar)", "radius (solar)", "orbit (AU)", "eccen.", "period (days)", "mass (Jupiter)", "radius (Jupiter)"]])
df

star                    \
                                 right asc. (deg) declination (deg)   
star                      planet                                      
11 Com                    b            185.179276         17.792868   
11 UMi                    b            229.274536         71.823898   
14 And                    b            352.822571         39.236198   
14 Her                    b            242.601303         43.817646   
16 Cyg B                  b            295.466553         50.517525   
18 Del                    b            314.608063         10.839286   
1RXS J160929.1-210524     b            242.376268        -21.083036   
24 Boo                    b            217.157547         49.844852   
24 Sex                    b            155.868210         -0.902244   
                          c            155.868210         -0.902244   
2MASS J01225093-2439505   b             20.712243        -24.664049   
2MASS J02192210-3925225   b             34.842106        -39.422928   
2MASS J04414489+2301513   b             70.437073         23.030941   
2MASS J12073346-3932539   b            181.889447        -39.548332   
2MASS J19383260+4603591   b            294.635865         46.066418   
2MASS J21402931+1625183 A b            325.122159         16.421759   
2MASS J22362452+4751425   b            339.102203         47.861820   
30 Ari B                  b             39.240585         24.648064   
4 UMa                     b            130.053406         64.327934   
42 Dra                    b            276.496399         65.563484   
47 UMa                    b            164.866562         40.430256   
                          c            164.866562         40.430256   
                          d            164.866562         40.430256   
51 Eri                    b             69.400551         -2.473548   
51 Peg                    b            344.366577         20.768833   
55 Cnc                    b            133.149216         28.330818   
                          c            133.149216         28.330818   
                          d            133.149216         28.330818   
                          e            133.149216         28.330818   
                          f            133.149216         28.330818   
...                                           ...               ...   
eps CrB                   b            239.396881         26.877880   
eps Eri                   b             53.232685         -9.458262   
eps Tau                   b             67.154167         19.180431   
gam 1 Leo                 b            154.993149         19.841488   
gam Cep                   b            354.836884         77.632278   
gam Lib                   b            233.881577        -14.789536   
                          c            233.881577        -14.789536   
iot Dra                   b            231.232391         58.966064   
kap And                   b            355.102112         44.333931   
kap CrB                   b            237.808044         35.657383   
mu Leo                    b            148.190903         26.006952   
nu Oph                    b            269.756622         -9.773632   
                          c            269.756622         -9.773632   
ome Ser                   b            237.573105          2.196509   
omi CrB                   b            230.035660         29.616209   
omi UMa                   b            127.566124         60.718170   
HD 39091                  c             84.291214        -80.469124   
psi 1 Dra B               b            265.492096         72.156914   
rho CrB                   b            240.261093         33.303509   
                          c            240.261093         33.303509   
tau Boo                   b            206.815598         17.456905   
tau Cet                   e             26.017012        -15.937480   
                          f             26.017012        -15.937480  

In [5]:
# DataFrame.unstack moves the sparse planet index into a dense set of columns.
# Every column (reduced to 2: orbit and mass) is duplicated 8 times because one star has 8 planets.

df[[("planet", "orbit (AU)"), ("planet", "mass (Jupiter)")]].unstack("planet")

planet                                      \
                           orbit (AU)                                       
planet                              b         c         d        e      f   
star                                                                        
11 Com                       1.290000       NaN       NaN      NaN    NaN   
11 UMi                       1.530000       NaN       NaN      NaN    NaN   
14 And                       0.830000       NaN       NaN      NaN    NaN   
14 Her                       2.930000       NaN       NaN      NaN    NaN   
16 Cyg B                     1.660000       NaN       NaN      NaN    NaN   
18 Del                       2.600000       NaN       NaN      NaN    NaN   
1RXS J160929.1-210524      330.000000       NaN       NaN      NaN    NaN   
24 Boo                       0.190000       NaN       NaN      NaN    NaN   
24 Sex                       1.333000  2.080000       NaN      NaN    NaN   
2MASS J01225093-2439505     52.000000       NaN       NaN      NaN    NaN   
2MASS J02192210-3925225    156.000000       NaN       NaN      NaN    NaN   
2MASS J04414489+2301513     15.000000       NaN       NaN      NaN    NaN   
2MASS J12073346-3932539     46.000000       NaN       NaN      NaN    NaN   
2MASS J19383260+4603591      0.920000       NaN       NaN      NaN    NaN   
2MASS J21402931+1625183 A         NaN       NaN       NaN      NaN    NaN   
2MASS J22362452+4751425    230.000000       NaN       NaN      NaN    NaN   
30 Ari B                     0.990000       NaN       NaN      NaN    NaN   
4 UMa                        0.870000       NaN       NaN      NaN    NaN   
42 Dra                       1.190000       NaN       NaN      NaN    NaN   
47 UMa                       2.100000  3.600000  11.60000      NaN    NaN   
51 Eri                      13.200000       NaN       NaN      NaN    NaN   
51 Peg                       0.052700       NaN       NaN      NaN    NaN   
55 Cnc                       0.115227  0.241376   5.50300  0.01544  0.788   
6 Lyn                        2.110000       NaN       NaN      NaN    NaN   
61 Vir                       0.050201  0.217500   0.47600      NaN    NaN   
7 CMa                        1.930000       NaN       NaN      NaN    NaN   
70 Vir                       0.481000       NaN       NaN      NaN    NaN   
75 Cet                       2.100000       NaN       NaN      NaN    NaN   
8 UMi                        0.490000       NaN       NaN      NaN    NaN   
81 Cet                       2.500000       NaN       NaN      NaN    NaN   
...                               ...       ...       ...      ...    ...   
XO-4                              NaN       NaN       NaN      NaN    NaN   
XO-5                         0.051500       NaN       NaN      NaN    NaN   
XO-6                         0.081500       NaN       NaN      NaN    NaN   
YZ Cet                       0.015570  0.020900   0.02764      NaN    NaN   
alf Ari                      1.200000       NaN       NaN      NaN    NaN   
alf Tau                      1.460000       NaN       NaN      NaN    NaN   
bet Cnc                      1.700000       NaN       NaN      NaN    NaN   
bet Pic                      9.100000       NaN       NaN      NaN    NaN   
bet UMi                      1.400000       NaN       NaN      NaN    NaN   
eps CrB                      1.300000       NaN       NaN      NaN    NaN   
eps Eri                      3.390000       NaN       NaN      NaN    NaN   
eps Tau                      1.930000       NaN       NaN      NaN    NaN   
gam 1 Leo                    1.190000       NaN       NaN      NaN    NaN   
gam Cep                      2.050000       NaN       NaN      NaN    NaN   
gam Lib                      1.240000  2.170000       NaN      NaN    NaN   
iot Dra                      1.275000       NaN       NaN      NaN    NaN   
kap And                     55.000000       NaN       NaN      NaN    NaN   
kap CrB                    

In [6]:
# We can also select a cross-section (xs) of the index by planet letter to focus on one at a time.

df.xs("b", level="planet")   # try "c", "d", "e", "f", "g", "h", "i"

star                                  \
                          right asc. (deg) declination (deg) distance (pc)   
star                                                                         
11 Com                          185.179276         17.792868         93.37   
11 UMi                          229.274536         71.823898        125.72   
14 And                          352.822571         39.236198         75.59   
14 Her                          242.601303         43.817646         17.94   
16 Cyg B                        295.466553         50.517525         21.41   
18 Del                          314.608063         10.839286         76.38   
1RXS J160929.1-210524           242.376268        -21.083036        145.00   
24 Boo                          217.157547         49.844852         96.25   
24 Sex                          155.868210         -0.902244         72.21   
2MASS J01225093-2439505          20.712243        -24.664049         36.00   
2MASS J02192210-3925225          34.842106        -39.422928         39.40   
2MASS J04414489+2301513          70.437073         23.030941        140.00   
2MASS J12073346-3932539         181.889447        -39.548332         52.40   
2MASS J19383260+4603591         294.635865         46.066418        396.36   
2MASS J21402931+1625183 A       325.122159         16.421759         25.00   
2MASS J22362452+4751425         339.102203         47.861820         74.00   
30 Ari B                         39.240585         24.648064         44.71   
4 UMa                           130.053406         64.327934         73.58   
42 Dra                          276.496399         65.563484         90.86   
47 UMa                          164.866562         40.430256         13.80   
51 Eri                           69.400551         -2.473548         29.40   
51 Peg                          344.366577         20.768833         15.47   
55 Cnc                          133.149216         28.330818         12.59   
6 Lyn                            97.696281         58.162632         54.74   
61 Vir                          199.601318        -18.311195          8.51   
7 CMa                            99.170990        -19.255878         19.82   
70 Vir                          202.107544         13.778788         17.91   
75 Cet                           38.039253         -1.034896         83.15   
8 UMi                           224.201477         74.900925        159.10   
81 Cet                           39.424168         -3.396175        101.53   
...                                    ...               ...           ...   
XO-3                             65.469581         57.817181        174.00   
XO-4                            110.388223         58.268108        293.00   
XO-5                            116.716527         39.094578        260.00   
XO-6                             94.793282         73.827682        237.06   
YZ Cet                           18.127668        -16.998966          3.60   
alf Ari                          31.793364         23.462423         20.21   
alf Tau                          68.980164         16.509302         20.43   
bet Cnc                         124.128838          9.185545         90.54   
bet Pic                          86.821198        -51.066513         19.75   
bet UMi                         222.676361         74.155502         38.78   
eps CrB                         239.396881         26.877880         69.98   
eps Eri                          53.232685         -9.458262          3.21   
eps Tau                          67.154167         19.180431         49.23   
gam 1 Leo                       154.993149         19.841488         38.52   
gam Cep                         354.836884         77.632278         13.54   
gam Lib                         233.881577        -14.789536         47.43   
iot Dra                         231.232391         58.966064         31.67   
kap And                         355.102112         44.333931

In [26]:
# Despite the tools Pandas provides, it can be easier to think of stars and planets as objects.

stardicts = []
for (starname, planetname), row in df.iterrows():
    if len(stardicts) == 0 or stardicts[-1]["name"] != starname:
        stardicts.append({"name": starname,
                          "ra": row["star", "right asc. (deg)"],
                          "dec": row["star", "declination (deg)"],
                          "dist": row["star", "distance (pc)"],
                          "mass": row["star", "mass (solar)"],
                          "radius": row["star", "radius (solar)"],
                          "planets": []})
    stardicts[-1]["planets"].append({"name": planetname,
                                     "orbit": row["planet", "orbit (AU)"],
                                     "eccen": row["planet", "eccen."],
                                     "period": row["planet", "period (days)"],
                                     "mass": row["planet", "mass (Jupiter)"],
                                     "radius": row["planet", "radius (Jupiter)"]})

stardicts[:30]

[{'name': '11 Com',
  'ra': 185.179276,
  'dec': 17.792868,
  'dist': 93.37,
  'mass': 2.7,
  'radius': 19.0,
  'planets': [{'name': 'b',
    'orbit': 1.29,
    'eccen': 0.231,
    'period': 326.03,
    'mass': 19.4,
    'radius': nan}]},
 {'name': '11 UMi',
  'ra': 229.27453599999998,
  'dec': 71.823898,
  'dist': 125.72,
  'mass': 2.78,
  'radius': 29.79,
  'planets': [{'name': 'b',
    'orbit': 1.53,
    'eccen': 0.08,
    'period': 516.21997,
    'mass': 14.74,
    'radius': nan}]},
 {'name': '14 And',
  'ra': 352.822571,
  'dec': 39.236198,
  'dist': 75.59,
  'mass': 2.2,
  'radius': 11.0,
  'planets': [{'name': 'b',
    'orbit': 0.83,
    'eccen': 0.0,
    'period': 185.84,
    'mass': 4.8,
    'radius': nan}]},
 {'name': '14 Her',
  'ra': 242.60130299999997,
  'dec': 43.817646,
  'dist': 17.94,
  'mass': 0.9,
  'radius': 0.93,
  'planets': [{'name': 'b',
    'orbit': 2.93,
    'eccen': 0.37,
    'period': 1773.40002,
    'mass': 4.66,
    'radius': nan}]},
 {'name': '16 Cyg B',


In [31]:
# But this destroys Numpy's array-at-a-time performance and (in some cases) convenience.

# Here's a way to get both (disclosure: I'm the author):
import awkward

stars = awkward.fromiter(stardicts)
stars

<Table [<Row 0> <Row 1> <Row 2> ... <Row 2932> <Row 2933> <Row 2934>] at 0x7f076640de48>

In [32]:
# The data are logically a collection of nested lists and dicts...

stars[:30].tolist()

[{'dec': 17.792868,
  'dist': 93.37,
  'mass': 2.7,
  'name': '11 Com',
  'planets': [{'eccen': 0.231,
    'mass': 19.4,
    'name': 'b',
    'orbit': 1.29,
    'period': 326.03,
    'radius': nan}],
  'ra': 185.179276,
  'radius': 19.0},
 {'dec': 71.823898,
  'dist': 125.72,
  'mass': 2.78,
  'name': '11 UMi',
  'planets': [{'eccen': 0.08,
    'mass': 14.74,
    'name': 'b',
    'orbit': 1.53,
    'period': 516.21997,
    'radius': nan}],
  'ra': 229.27453599999998,
  'radius': 29.79},
 {'dec': 39.236198,
  'dist': 75.59,
  'mass': 2.2,
  'name': '14 And',
  'planets': [{'eccen': 0.0,
    'mass': 4.8,
    'name': 'b',
    'orbit': 0.83,
    'period': 185.84,
    'radius': nan}],
  'ra': 352.822571,
  'radius': 11.0},
 {'dec': 43.817646,
  'dist': 17.94,
  'mass': 0.9,
  'name': '14 Her',
  'planets': [{'eccen': 0.37,
    'mass': 4.66,
    'name': 'b',
    'orbit': 2.93,
    'period': 1773.40002,
    'radius': nan}],
  'ra': 242.60130299999997,
  'radius': 0.93},
 {'dec': 50.517525,
  

In [53]:
# ...but they have been entirely converted into arrays.
for starattr in "name", "ra", "dec", "dist", "mass", "radius":
    print("{:15s} =".format("stars[{!r:}]".format(starattr)), stars[starattr])

print()
for planetattr in "name", "orbit", "eccen", "period", "mass", "radius":
    print("{:26s} =".format("stars['planets'][{!r:}]".format(planetattr)), stars["planets"][planetattr])

stars['name']   = ['11 Com' '11 UMi' '14 And' ... 'tau Gem' 'ups And' 'xi Aql']
stars['ra']     = [185.179276 229.274536 352.822571 ... 107.784882  24.199345 298.562012]
stars['dec']    = [17.792868 71.823898 39.236198 ... 30.245163 41.40546   8.461452]
stars['dist']   = [ 93.37 125.72  75.59 ... 112.64  13.41  56.27]
stars['mass']   = [2.7  2.78 2.2  ... 2.3  1.3  2.2 ]
stars['radius'] = [19.   29.79 11.   ... 26.8   1.56 12.  ]

stars['planets']['name']   = [['b'] ['b'] ['b'] ... ['b'] ['b' 'c' 'd'] ['b']]
stars['planets']['orbit']  = [[1.29] [1.53] [0.83] ... [1.17] [0.059222000000000004 0.827774 2.51329] [0.68]]
stars['planets']['eccen']  = [[0.231] [0.08] [0.0] ... [0.031] [0.0215 0.2596 0.2987] [0.0]]
stars['planets']['period'] = [[326.03] [516.21997] [185.84] ... [305.5] [4.617033 241.258 1276.46] [136.75]]
stars['planets']['mass']   = [[19.4] [14.74] [4.8] ... [20.6] [0.6876 1.981 4.132] [2.8]]
stars['planets']['radius'] = [[nan] [nan] [nan] ... [nan] [nan nan nan] [nan]]
